In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import sys

import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [4]:
#__________________Path Builder for DOCS__________________
root_path = '/Users/davide_lupis/Desktop/20_newsgroup/'
arg_path = root_path+os.listdir(root_path)[0]
doc_path = arg_path + '/'+os.listdir(arg_path)[0]
#---------------------------------------------------------

#___________________ Feature in TEXT _____________________
with open(doc_path, 'r') as f:
    text = f.read()
    print('Language: ', nlp(text).lang_)
    
doc = nlp(text)
features = []
for token in doc:
    features.append({'token' : token.text,
                     'pos' : token.pos_ ,
                     'lamma':token.lemma_
                    })
df = pd.DataFrame(features)
#---------------------------------------------------------
df

Language:  en


,token,pos,lamma
0,Newsgroups,NOUN,newsgroup
1,:,PUNCT,:
2,talk.politics.mideast,PROPN,talk.politics.mideast
3,\n,SPACE,\n
4,Path,NOUN,path
...,...,...,...
301,.,PUNCT,.
302,\n\n,SPACE,\n\n
303,Harry,PROPN,Harry
304,.,PROPN,.
